<a href="https://colab.research.google.com/github/amalinc/Face-Recognition/blob/main/MultyLabel_FineTuning_Gemma3_270m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# התקנת הספריות הנדרשות
!pip install -q transformers accelerate bitsandbytes peft scipy scikit-learn pandas openpyxl datasets

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset

In [30]:
# --- חלק א': טעינת נתונים והכנה ---

# 1. טעינת הנתונים: קובץ האימון/אימות וקובץ הבדיקה
df_train_val = pd.read_excel('train_final.xlsx')
df_test = pd.read_excel('test_final.xlsx')

# נניח שמות עמודות:
TEXT_COLUMN = 'text'
CATEGORIES_COLUMN = 'labels'
ID_COLUMN = 'ID' # 💡 הוספת עמודת ה-ID

# 2. הפרדת טקסט, תוויות ו-IDs עבור שני הקבצים
texts_train_val = df_train_val[TEXT_COLUMN].tolist()
labels_train_val_raw = df_train_val[CATEGORIES_COLUMN].tolist()
ids_train_val = df_train_val[ID_COLUMN].tolist() # טעינת ה-ID לאימון/אימות

texts_test = df_test[TEXT_COLUMN].tolist()
labels_test_raw = df_test[CATEGORIES_COLUMN].tolist()
ids_test = df_test[ID_COLUMN].tolist() # טעינת ה-ID הבדיקה

# 3. המרה לרשימת רשימות של קטגוריות באמצעות המפריד '|'
# Handle potential NaN values by converting to string, split by '|', and strip whitespace from each label
labels_list_train_val = [[label.strip() for label in str(c).split('|')] for c in labels_train_val_raw]
labels_list_test = [[label.strip() for label in str(c).split('|')] for c in labels_test_raw]


# Combine labels from both train/val and test for fitting the MultiLabelBinarizer
all_labels = labels_list_train_val + labels_list_test

# 4. הצפנה בינארית (Multi-Hot Encoding)
# חשוב: fit על נתוני האימון/אימות בלבד, transform על כולם
mlb = MultiLabelBinarizer()
mlb.fit(all_labels) # Fit on combined and cleaned labels
labels_encoded_train_val = mlb.transform(labels_list_train_val)
labels_encoded_test = mlb.transform(labels_list_test) # שימוש באותם סיווגים


NUM_LABELS = len(mlb.classes_)
print(f"שמות {NUM_LABELS} הקטגוריות: {mlb.classes_}")

# 5. חלוקת קובץ האימון/אימות ל-80% אימון (Train) ו-20% אימות (Validation)
X_train, X_val, y_train, y_val, ids_train, ids_val = train_test_split(
    texts_train_val, labels_encoded_train_val, ids_train_val, test_size=0.2, random_state=42
)

# הגדרת סט הבדיקה (Test) מהקובץ השני
X_test, y_test = texts_test, labels_encoded_test # ids_test is already available


print(f"גודל סט אימון: {len(X_train)} (80% מ-500)")
print(f"גודל סט אימות: {len(X_val)} (20% מ-500)")
print(f"גודל סט בדיקה: {len(X_test)} (100 מאמרים)")

שמות 11 הקטגוריות: ['בריאות ואורח חיים' 'דיור וקהילה' 'זקנה והזדקנות' 'חברה ותרבות'
 'חינוך והשכלה' 'כלכלה ותעסוקה' 'משפחה ויחסים' 'משפט ומדיניות' 'נגישות'
 'נתונים' 'תיאוריות וגישות']
גודל סט אימון: 400 (80% מ-500)
גודל סט אימות: 100 (20% מ-500)
גודל סט בדיקה: 100 (100 מאמרים)


In [31]:
import pandas as pd
from IPython.display import display

# Create a pandas DataFrame from X_val for nice display
# Assuming X_val is a list of strings
if isinstance(X_val, list):
    df_x_val = pd.DataFrame(X_val)
    display(df_x_val)
else:
    print("X_val is not a list and cannot be displayed as a DataFrame.")
    print(X_val) # Fallback to standard print

,0
0,כותרת: אנשים עם מוגבלויות – בין זכויות חברתיות...
1,"כותרת: מציאות משותפת מתמשכת - טראומה, צמיחה וח..."
2,כותרת: איזון בית-עבודה בקרב הורים לילדים עם מו...
3,כותרת: לראות אדם : תמיכות במסע החיים של מבוגרי...
4,"כותרת: מכשירי שיקום וניידות: מימוש הזכאות, השי..."
...,...
95,כותרת: אנשים עם מוגבלות בישראל: עובדות ומספרים...
96,כותרת: מעמסה וצמיחה בקרב אימהות למתבגרים עם מו...
97,כותרת: מפגש נשים עם מוגבלות עם מערכת הבריאות ב...
98,כותרת: Extra Fragile in Disaster –People with ...


In [5]:
HF_TOKEN = userdata.get("HF_TOKEN")

In [32]:
import torch
from transformers import AutoTokenizer, GemmaForSequenceClassification, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import f1_score, accuracy_score
from datasets import Dataset
from google.colab import userdata
import numpy as np # Import numpy

# ----------------- 1. טעינת המודל והטוקנייזר (ללא שינוי) -----------------
MODEL_ID = "google/gemma-3-270m"
# HF_TOKEN = "hf_..." # החלף ב-Token שלך (אם נדרש)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

# ודא ש-NUM_LABELS ו-mlb קיימים מהכנת הנתונים
model = GemmaForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=NUM_LABELS,
    id2label={i: c for i, c in enumerate(mlb.classes_)},
    label2id={c: i for c, i in enumerate(mlb.classes_)},
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)
model.config.problem_type = "multi_label_classification"
model.config.pad_token_id = tokenizer.eos_token_id

# ----------------- 2. הגדרת LoRA והכנת המודל (ללא שינוי) -----------------
lora_config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05, bias="none", task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model, lora_config)
print("--- פרמטרים לאימון LoRA ---")
model.print_trainable_parameters()


# ----------------- 3. הכנת Dataset ופונקציות (תיקון סופי ל-dtype) -----------------

# פונקציית טוקניזציה
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# יצירת Datasets
# Ensure labels are in a list format for the Dataset.from_dict
train_data = Dataset.from_dict({'text': X_train, 'labels': y_train.tolist()})
val_data = Dataset.from_dict({'text': X_val, 'labels': y_val.tolist()})

tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_val_data = val_data.map(tokenize_function, batched=True)

# --- Define Custom Data Collator to handle label dtype ---
class CustomDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        # Use the standard DataCollatorWithPadding to handle text inputs (padding, etc.)
        batch = super().__call__(features)

        # Ensure labels are present and convert them to Float32 torch tensors
        if "labels" in batch:
            # Convert numpy array from dataset to torch tensor and cast to float32
            batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)

        return batch

# Initialize the custom data collator
custom_data_collator = CustomDataCollator(tokenizer=tokenizer)

# 3.1 פונקציית חישוב המדדים שלך (compute_metrics) - הועברה לכאן
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # ודא שה-logits וה-labels הם באותו dtype לפני הפעלת Sigmoid
    logits = torch.Tensor(logits).float() # המרה ל-Float32
    labels = torch.Tensor(labels).float().numpy() # המרה ל-Float32 ו-Numpy

    probs = torch.sigmoid(logits)
    y_pred = (probs >= 0.5).int().numpy()

    f1_macro = f1_score(labels, y_pred, average='macro', zero_division=0)
    exact_match_ratio = accuracy_score(labels, y_pred)
    return {'f1_macro': f1_macro, 'exact_match_ratio': exact_match_ratio}


# 4. הגדרת פרמטרי האימון וה-Trainer
training_args = TrainingArguments(
    output_dir="./gemma_270m_multi_label_results",
    num_train_epochs=5, per_device_train_batch_size=8, per_device_eval_batch_size=8,
    learning_rate=2e-5, weight_decay=0.01, eval_strategy="epoch", save_strategy="epoch",
    load_best_model_at_end=True, metric_for_best_model="f1_macro",
    report_to="none" # Disable reporting to external services like wandb
)

trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data, tokenizer=tokenizer, compute_metrics=compute_metrics,
    data_collator=custom_data_collator # Use the custom data collator
)

# 5. התחל אימון
print("--- מתחיל אימון... זה עשוי לקחת זמן ---")
trainer.train()

You are using a model of type gemma3_text to instantiate a model of type gemma. This is not supported for all configurations of models and can yield errors.
Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-3-270m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- פרמטרים לאימון LoRA ---
trainable params: 3,804,032 || all params: 271,876,992 || trainable%: 1.3992


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/tmp/ipython-input-3720945115.py:97: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


--- מתחיל אימון... זה עשוי לקחת זמן ---


/tmp/ipython-input-3720945115.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)


Epoch,Training Loss,Validation Loss,F1 Macro,Exact Match Ratio
1,No log,4.578800,0.181514,0.000000
2,No log,3.767310,0.185732,0.000000
3,No log,3.139351,0.196107,0.020000
4,No log,3.013292,0.168522,0.020000
5,No log,3.298674,0.127348,0.020000


You are using a model of type gemma3_text to instantiate a model of type gemma. This is not supported for all configurations of models and can yield errors.
/tmp/ipython-input-3720945115.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)
You are using a model of type gemma3_text to instantiate a model of type gemma. This is not supported for all configurations of models and can yield errors.
/tmp/ipython-input-3720945115.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float32)
You are using a model of type gemma3_text to instantiate a model of type gemma

TrainOutput(global_step=250, training_loss=4.005214599609375, metrics={'train_runtime': 69.5547, 'train_samples_per_second': 28.754, 'train_steps_per_second': 3.594, 'total_flos': 319810043904000.0, 'train_loss': 4.005214599609375, 'epoch': 5.0})

In [33]:
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset # נדרש ליצירת מבנה הנתונים
import pandas as pd # Import pandas for displaying results

print("--- סיום אימון, עובר להערכה ידנית סופית (עקיפת שגיאות פורמט) ---")

# ודא שפונקציית הטוקניזציה זמינה (נלקחה מחלק ב')
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# 1. יצירת Dataset והמרת הנתונים ל-tensors בצורה מפורשת
test_data = Dataset.from_dict({'text': X_test, 'labels': y_test.tolist(), 'id': ids_test})
tokenized_test_data = test_data.map(tokenize_function, batched=True) # משתמש ב-tokenize_function מחלק ב'

# המרה מפורשת ומוכחת של ה-dtypes: input_ids/attention_mask: Long, labels: Float32
input_ids_test = torch.tensor(tokenized_test_data['input_ids'], dtype=torch.long)
attention_mask_test = torch.tensor(tokenized_test_data['attention_mask'], dtype=torch.long)
labels_test_tensor = torch.tensor(tokenized_test_data['labels'], dtype=torch.float32) # Rename to avoid conflict with numpy labels

# 2. יצירת DataLoader וביצוע חיזויים
test_dataset = TensorDataset(input_ids_test, attention_mask_test, labels_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=8)

model.eval() # מעבר למצב הערכה
all_logits = []
all_labels = []

# Determine the device from the model
device = model.device if hasattr(model, 'device') else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))


with torch.no_grad():
    for input_ids, attention_mask, labels in test_dataloader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # 💡 התיקון הסופי: המרת BFloat16 ל-Float32 לפני המעבר ל-NumPy
        all_logits.append(outputs.logits.cpu().to(torch.float32).numpy())
        all_labels.append(labels.cpu().numpy())

# 3. חישוב מדדים סופיים
logits = np.concatenate(all_logits, axis=0)
labels = np.concatenate(all_labels, axis=0)

probs = 1 / (1 + np.exp(-logits)) # פונקציית Sigmoid
y_pred = (probs >= 0.5).astype(int)

f1_macro = f1_score(labels, y_pred, average='macro', zero_division=0)
exact_match_ratio = accuracy_score(labels, y_pred)

print("--- תוצאות הערכה סופיות על סט הבדיקה ---")
print(f"F1-Macro: {f1_macro:.4f}")
print(f"Exact Match Ratio: {exact_match_ratio:.4f}")

# 4. הצגת החיזויים
print("\n--- הצגת חיזויים ---")

# המרת התוויות המקודדות חזרה לשמות קטגוריות
true_labels_names = mlb.inverse_transform(labels)
predicted_labels_names = mlb.inverse_transform(y_pred)

# --- Diagnostic Prints ---
print("\n--- Diagnostic Info ---")
print(f"MLB Classes: {mlb.classes_}")
print(f"Shape of labels (y_test): {labels.shape}")
print(f"Sample of true_labels_names (first 5): {true_labels_names[:5]}")
print(f"Sample of y_test (first 5 rows): \n{labels[:5]}")
# --- End Diagnostic Prints ---

# יצירת DataFrame להצגה נוחה
results_df = pd.DataFrame({
    'ID': ids_test,
    'Text': X_test,
    'True Labels': [', '.join(map(str, l)) for l in true_labels_names], # Join lists for display
    'Predicted Labels': [', '.join(map(str, l)) for l in predicted_labels_names] # Join lists for display
})

display(results_df.head()) # הצגת 5 השורות הראשונות

# 5. שמירת החיזויים לקובץ אקסל
output_excel_path = "./predictions.xlsx"
results_df.to_excel(output_excel_path, index=False)
print(f"\n✅ החיזויים נשמרו לקובץ: {output_excel_path}")


# 6. שמירת המודל המאומן הטוב ביותר
output_dir = "./final_gemma_270m_model"
# Check if trainer exists before saving using it
if 'trainer' in locals() and trainer is not None:
    trainer.save_model(output_dir)
elif 'model' in locals() and model is not None:
     # If trainer is not available, save the model directly
     model.save_pretrained(output_dir)
     # If it's a PEFT model, this saves the adapter weights
     # If it's the base model, it saves the whole model
else:
    print("Warning: Neither trainer nor model object found to save.")

# Save tokenizer as well, as it's needed for inference
if 'tokenizer' in locals() and tokenizer is not None:
    tokenizer.save_pretrained(output_dir)
else:
    print("Warning: Tokenizer object not found to save.")

print(f"✅ המודל (וה-LoRA Adapter) נשמר בהצלחה ב: {output_dir}")

--- סיום אימון, עובר להערכה ידנית סופית (עקיפת שגיאות פורמט) ---


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

--- תוצאות הערכה סופיות על סט הבדיקה ---
F1-Macro: 0.2247
Exact Match Ratio: 0.0400

--- הצגת חיזויים ---

--- Diagnostic Info ---
MLB Classes: ['בריאות ואורח חיים' 'דיור וקהילה' 'זקנה והזדקנות' 'חברה ותרבות'
 'חינוך והשכלה' 'כלכלה ותעסוקה' 'משפחה ויחסים' 'משפט ומדיניות' 'נגישות'
 'נתונים' 'תיאוריות וגישות']
Shape of labels (y_test): (100, 11)
Sample of true_labels_names (first 5): [('בריאות ואורח חיים',), ('דיור וקהילה', 'חינוך והשכלה'), ('בריאות ואורח חיים', 'דיור וקהילה'), ('חינוך והשכלה', 'תיאוריות וגישות'), ('חברה ותרבות',)]
Sample of y_test (first 5 rows): 
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


,ID,Text,True Labels,Predicted Labels
0,test_001,כותרת: תשומת לב ללב בחירום - תובנות מחדר הטיפו...,בריאות ואורח חיים,
1,test_002,כותרת: תרומתן של ההשכלה האקדמית ושל תוכניות הה...,"דיור וקהילה, חינוך והשכלה",
2,test_003,כותרת: תרומת השיקום בקהילה לצמצום היקף האשפוז ...,"בריאות ואורח חיים, דיור וקהילה",דיור וקהילה
3,test_004,כותרת: תרבות של הידברות - דיאלוג בין מרצים לבי...,"חינוך והשכלה, תיאוריות וגישות","חינוך והשכלה, משפט ומדיניות, נגישות"
4,test_005,כותרת: תקשורת וקולנוע ככלי להעצמה | תקציר: המא...,חברה ותרבות,



✅ החיזויים נשמרו לקובץ: ./predictions.xlsx


You are using a model of type gemma3_text to instantiate a model of type gemma. This is not supported for all configurations of models and can yield errors.


✅ המודל (וה-LoRA Adapter) נשמר בהצלחה ב: ./final_gemma_270m_model


In [12]:
import shutil
import os

output_dir = "./final_gemma_270m_model"
zip_output_path = "./final_gemma_270m_model.zip"

if os.path.exists(output_dir):
    shutil.make_archive(output_dir, 'zip', output_dir)
    print(f"✅ Directory '{output_dir}' zipped to '{zip_output_path}'")
else:
    print(f"❌ Directory '{output_dir}' not found. Please ensure the model was saved correctly.")

✅ Directory './final_gemma_270m_model' zipped to './final_gemma_270m_model.zip'


After running the cell above, you should see a file named `final_gemma_270m_model.zip` in your Colab file browser. You can then download this zip file to your local computer by right-clicking on it in the file browser and selecting "Download".